In [1]:
def _fix_signal():
    """Unregister PETSc's problematic signal handling after import

    By restoring IPython's default signal handlers
    
    and also fix duplicate log handlers registered by importing ffcx
    """
    from traitlets.config import Application

    Application.instance().init_signal()


def apply_imports(rc):
    """Synchronize imports on both the notebook and all engines"""
    with rc[:].sync_imports():
        from mpi4py import MPI
        import dolfinx
        import time
        import ufl
        import numpy

    rc[:].apply_sync(_fix_signal)
    
# avoid duplicate logs after importing ffcx:
# https://github.com/ipython/ipyparallel/pull/797
import logging 
logging.getLogger("ipyparallel.cluster").propagate = False

In [2]:
def solve_poisson(N):
    """Solves the poisson equation on an NxN unit square.

    Returns the time taken to compute the solution
    """
    MPI.COMM_WORLD.Barrier()
    start = time.perf_counter()
    mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, N, N)
    V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    x = ufl.SpatialCoordinate(mesh)
    f = ufl.cos(2 * ufl.pi * x[0])
    L = ufl.inner(f, v) * ufl.dx
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    bndry_facets = dolfinx.mesh.exterior_facet_indices(mesh.topology)
    bndry_dofs = dolfinx.fem.locate_dofs_topological(
        V, mesh.topology.dim - 1, bndry_facets
    )
    bc = [dolfinx.fem.dirichletbc(numpy.float64(0), bndry_dofs, V)]
    petsc_options = {
        "ksp_type": "cg",
        "pc_type": "hypre",
        "pc_hypre_type": "boomeramg",
        "ksp_rtol": 1.0e-8,
        "pc_hypre_boomeramg_strong_threshold": 0.7,
        "pc_hypre_boomeramg_agg_nl": 4,
        "pc_hypre_boomeramg_agg_num_paths": 2,
    }

    problem = dolfinx.fem.petsc.LinearProblem(a, L, bc, petsc_options=petsc_options)
    uh = problem.solve()
    end = time.perf_counter()

    local_cntr = dolfinx.fem.assemble_scalar(dolfinx.fem.form(uh * uh * ufl.dx))
    global_cntr = mesh.comm.reduce(local_cntr, op=MPI.SUM, root=0)
    it = problem.solver.getIterationNumber()
    if mesh.comm.rank == 0:
        print(
            f"Num processes: {mesh.comm.size} Num dofs: {V.dofmap.index_map.size_global} Num iterations: {it} int u^2: {global_cntr}"
        )
    return end - start

In [3]:
import ipyparallel as ipp

with ipp.Cluster(n=1, engines="mpi", log_level=30) as rc:
    apply_imports(rc)
    result_serial = rc[:].apply_async(solve_poisson, 300)
    with result_serial.stream_output():
        result_serial.wait_interactive()

  0%|          | 0/1 [00:00<?, ?engine/s]

importing MPI from mpi4py on engine(s)
importing dolfinx on engine(s)
importing time on engine(s)
importing ufl on engine(s)
importing numpy on engine(s)


solve_poisson:   0%|          | 0/1 [00:00<?, ?tasks/s]

[stdout:0] Num processes: 1 Num dofs: 90601 Num iterations: 9 int u^2: 0.00021415468086532634


In [4]:
import numpy

from tqdm.notebook import tqdm

mpi_processes = [1, 2, 4, 8]

measurements = []

for i, processes in enumerate(tqdm(mpi_processes, desc="overall progress")):
    cluster = ipp.Cluster(n=processes, engines="mpi", log_level=30)
    with cluster as rc:
        apply_imports(rc)
        process_runtimes = rc[:].apply_async(solve_poisson, 600)
        with process_runtimes.stream_output():
            process_runtimes.wait_interactive()
        for t in process_runtimes:
            measurements.append({"t": t, "processes": processes})

overall progress:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?engine/s]

importing MPI from mpi4py on engine(s)
importing dolfinx on engine(s)
importing time on engine(s)
importing ufl on engine(s)
importing numpy on engine(s)


solve_poisson:   0%|          | 0/1 [00:00<?, ?tasks/s]

[stdout:0] Num processes: 1 Num dofs: 361201 Num iterations: 9 int u^2: 0.00021416720338638073


...

[stdout:0] Num processes: 4 Num dofs: 361201 Num iterations: 16 int u^2: 0.00021416720357179264


  0%|          | 0/8 [00:00<?, ?engine/s]

importing MPI from mpi4py on engine(s)
importing dolfinx on engine(s)
importing time on engine(s)
importing ufl on engine(s)
importing numpy on engine(s)


solve_poisson:   0%|          | 0/8 [00:00<?, ?tasks/s]

[stdout:0] Num processes: 8 Num dofs: 361201 Num iterations: 17 int u^2: 0.0002141672031712013


We can collect our timing measurements to inspect our parallel performance.
We have a measurement for every worker, so we can see if there are any outliers going idle or doing extra work.

In [5]:
import pandas as pd
import altair as alt

df = pd.DataFrame(measurements)
df.groupby("processes").t.mean()

processes
1    1.397290
2    1.263056
4    1.065009
8    0.859183
Name: t, dtype: float64

We can compute parallel 'speedup' by comparing durations with the n=1 case.

In [6]:
serial_time = df[df.processes == 1].t[0]
df["speedup"] = serial_time / df["t"]

In [7]:
alt.Chart(df).mark_point().encode(
    x="processes",
    y="t",
    color="speedup",
    tooltip=[
        "processes",
        "t",
        "speedup",
    ],
).interactive()

alt.Chart(...)